AI Health & Fitness Plan Generator (LangChain)
📌 Problem Statement
Build a Google Colab notebook that generates a personalized health + fitness plan using LangChain and an LLM.

The notebook should:

Collect user details (age, weight, height, activity level, diet preference, fitness goal)

Use a LangChain multi-step workflow to generate:

🍽️ Meal plan with macros (protein/carbs/fats)
🏃 Workout plan with warm-up, main workout, cool-down
📅 Weekly combined strategy with schedule + motivation tips
Display results neatly inside Colab (Markdown-style output is enough)

Export the final plan to a downloadable JSON and/or Markdown file

🧩 Core Features
1) Input Collection in Colab
Students must take input using one of these:

Python input() prompts inside Colab, OR
A Python dictionary / JSON block in a cell (recommended)
Example (recommended):

user_profile = {
  "age": 28,
  "weight_kg": 72,
  "height_cm": 175,
  "activity_level": "moderate",
  "diet_preference": "vegetarian",
  "fitness_goal": "fat loss"
}
2) Use LangChain for Orchestration
Create 3 LangChain chains (or one router chain + 3 subchains):

Meal Plan Chain
Workout Plan Chain
Weekly Strategy Chain (combines outputs from first two)
3) Structured Output (Must)
Use PydanticOutputParser (or JSON parser) so outputs are consistent and machine-readable.

4) Display + Export
Display final output in Colab clearly

Save output to:

final_plan.json
(optional) final_plan.md
🚫 Technical Constraints
✅ Plain Python in Colab

✅ Must use LangChain

 ✅ Must use .env OR Colab userdata/manual key paste (safe approach)

 ✅ Must run end-to-end in a single notebook

🛠️ Tech Stack (Colab)
Component	Technology
Runtime	Google Colab
Language	Python
Orchestration	LangChain
LLM	OpenAI (via LangChain)
Output validation	Pydantic
Export	JSON / Markdown
✅ Expected Student Deliverables
One Colab Notebook (.ipynb) that:
installs dependencies
loads API key
collects user profile
runs LangChain workflow
prints final plan
exports result to file(s)
A short README section inside the notebook (top cell) explaining:
setup steps
where to put API key
example input
what files are generated
📒 Suggested Notebook Cell Flow (What you enforce)
Title + problem statement (Markdown cell)
Install dependencies (pip install ...)
Import libraries
Load API key securely
Define schema (Pydantic models)
Define prompts
Build LangChain pipeline
Run with sample input
Display output
Save JSON/MD

In [8]:
!pip install -q langchain langchain-google-genai

In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import json
from datetime import datetime


llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite",google_api_key=userdata.get('Assignments'))
print("Libraries imported and API key Added")


Libraries imported and API key Added


In [18]:
user_profile = {
  "age": 28,
  "weight_kg": 72,
  "height_cm": 175,
  "activity_level": "moderate",
  "diet_preference": "vegetarian",
  "fitness_goal": "fat loss"
}
meal_prompt = PromptTemplate(
    input_variables=["age", "weight_kg", "height_cm", "activity_level", "diet_preference", "fitness_goal"],
    template="""You are an expert nutritionist and health expert and also fitness expert. Create a personalized daily meal plan.

User Details:
- Age: {age} years
- Weight: {weight_kg} kg
- Height: {height_cm} cm
- Activity Level: {activity_level}
- Diet Preference: {diet_preference}
- Fitness Goal: {fitness_goal}

Create a meal plan with 3-4 meals including:
- Meal name (Breakfast, Lunch, Dinner, Snack)
- Food items
- Calories and macros (protein, carbs, fats in grams)
- Preparation tips

Format as clear sections. Include daily totals and hydration advice."""
)
workout_prompt = PromptTemplate(
    input_variables=["age", "weight_kg", "activity_level", "fitness_goal"],
    template="""You are an expert fitness trainer. Create a personalized workout plan.

User Details:
- Age: {age} years
- Weight: {weight_kg} kg
- Activity Level: {activity_level}
- Fitness Goal: {fitness_goal}

Create a complete workout session with:
1. WARM-UP (5-10 min): Include 3-4 exercises with duration
2. MAIN WORKOUT (20-40 min): Include 5-7 exercises with sets, reps, and rest periods
3. COOL-DOWN (5-10 min): Include 3-4 stretches with duration

Add total duration, intensity level, and required equipment."""
)

strategy_prompt = PromptTemplate(
    input_variables=["age", "fitness_goal", "activity_level", "meal_summary", "workout_summary"],
    template="""You are a wellness coach. Create a 7-day integrated health strategy.

User Details:
- Age: {age} years
- Fitness Goal: {fitness_goal}
- Activity Level: {activity_level}

Meal Plan Summary: {meal_summary}
Workout Plan Summary: {workout_summary}

Create a 7-day schedule with:
- Each day (Monday to Sunday)
- Workout type and focus area
- Meal/nutrition focus
- Daily motivation tip

Also include:
- Weekly calorie target
- Progression plan for future weeks
- 3-5 success metrics to track
- Motivational quote"""
)
meal_chain = meal_prompt | llm
workout_chain = workout_prompt | llm
strategy_chain = strategy_prompt | llm

print("Your Meal Plan")
meal_plan = meal_chain.invoke(user_profile).content
print(meal_plan)
print("Your Workout Plan")
workout_plan = workout_chain.invoke(user_profile).content
print(workout_plan)
meal_summary = f"Daily nutrition plan for {user_profile['diet_preference']} diet targeting {user_profile['fitness_goal']}"
workout_summary = f"Workout plan for {user_profile['activity_level']} activity level"
strategy_input = {
    "age": user_profile["age"],
    "fitness_goal": user_profile["fitness_goal"],
    "activity_level": user_profile["activity_level"],
    "meal_summary": meal_summary,
    "workout_summary": workout_summary
}
print("Your Weekly Strategy")
strategy_plan = strategy_chain.invoke(strategy_input).content
print(strategy_plan)

Your Meal Plan
Here's a personalized daily meal plan designed for a 28-year-old vegetarian male, 72kg, 175cm, with a moderate activity level, aiming for fat loss. This plan focuses on nutrient-dense foods, adequate protein for satiety and muscle preservation, and controlled calorie intake.

**Important Note:** This is a general guideline. Individual calorie and macro needs can vary. It's highly recommended to consult with a registered dietitian or nutritionist for a truly personalized plan, especially if you have any underlying health conditions.

---

## Personalized Vegetarian Fat Loss Meal Plan

**User Profile:**
*   **Age:** 28 years
*   **Weight:** 72 kg
*   **Height:** 175 cm
*   **Activity Level:** Moderate
*   **Diet Preference:** Vegetarian
*   **Fitness Goal:** Fat Loss

---

### Estimated Daily Calorie and Macronutrient Target for Fat Loss:

Based on your details and moderate activity level, a reasonable starting point for fat loss is approximately **1800-2000 calories**. Th

In [20]:
plan_data = {
    "generated_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "user_profile": user_profile,
    "meal_plan": meal_plan,
    "workout_plan": workout_plan,
    "weekly_strategy": strategy_plan
}
json_filename = "final_plan.json"
with open(json_filename, 'w', encoding='utf-8') as f:
    json.dump(plan_data, f, indent=2, ensure_ascii=False)

In [22]:
from google.colab import files
files.download('final_plan.json')
print("File Downloaded")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File Downloaded
